In [92]:
pip install catboost

In [131]:
#import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression as Linr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from catboost import CatBoostRegressor as cbr
from catboost import Pool
#from bayes_opt import BayesianOptimization

In [94]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
df = pd.read_csv('/content/drive/MyDrive/Datathon 2024/Chevron/Train/training.csv')
df

,Unnamed: 0,surface_x,surface_y,bh_x,bh_y,standardized_operator_name,gross_perforated_length,number_of_stages,total_proppant,total_fluid,...,relative_well_position,batch_frac_classification,well_family_relationship,frac_type,frac_seasoning,horizontal_midpoint_x,horizontal_midpoint_y,horizontal_toe_x,horizontal_toe_y,OilPeakRate
0,0,1383493.751,717329.2368,1382854.564,712767.3700,1121,3963.582677,NaN,76000.0000,NaN,...,Standalone Well,Unknown,Standalone Well,Primary Frac,NaN,1383030.678,714796.3610,1382668.912,712798.4321,46.623023
1,1,1380344.035,711916.2732,1379738.429,707860.5643,1022,3179.133858,NaN,113000.0000,5.610964e+05,...,Outer Well,Unknown,Infill Child Well,Primary Frac,NaN,1379965.126,709541.7145,1379630.025,707907.6708,59.750009
2,2,1379915.895,717845.0063,1379274.644,713471.9952,1304,3810.039370,12.0,286571.4286,1.621424e+06,...,Standalone Well,Non-Batch Frac,Standalone Well,Primary Frac,92.0,1379357.250,715381.2327,1378960.372,713530.4490,10.785716
3,3,1383567.443,706640.2010,1382891.477,702068.3838,1022,3723.425197,11.0,106878.9286,4.377134e+05,...,Standalone Well,Unknown,Standalone Well,Primary Frac,NaN,1383099.610,704058.8508,1382727.299,702098.4882,123.797638
4,4,1378962.585,702623.9787,1379643.546,707204.9131,1022,3811.023622,NaN,112789.7143,6.229802e+05,...,Outer Well,Unknown,Infill Child Well,Primary Frac,NaN,1379546.414,705150.7519,1379987.487,707123.7258,102.309540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29687,30087,1441088.593,978921.8705,1432758.032,979984.5869,1050,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN
29688,30088,1513299.189,976157.4486,1519890.072,975148.4820,1159,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN
29689,30089,1458652.142,972862.2346,1453129.889,973195.0998,1050,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN
29690,30090,1463417.436,972485.8276,1468807.765,971564.9743,1153,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Primary Frac,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#remove irrelavant columns based on correlation and quantum of null values
def remove_columns(df):
  columns_to_remove = ['Unnamed: 0', 'bh_x', 'bh_y', 'standardized_operator_name',
                       'number_of_stages', 'total_proppant', 'average_stage_length',
                       'average_proppant_per_stage', 'average_frac_fluid_per_stage',
                       'pad_id',  'horizontal_midpoint_x', 'horizontal_midpoint_y',
                       'horizontal_toe_x', 'horizontal_toe_y', 'frac_type']
  return df.drop(columns_to_remove, axis=1)
df1 = remove_columns(df)

In [97]:
#remove rows where OilPeaks is Nan
def remove_target_nan(df):
  return df.loc[df['OilPeakRate'].isnull() == False]

df2 = remove_target_nan(df1)

In [98]:
#remove outliers
def clean_data(df):
  row1 = df[(df["proppant_intensity"]>4000)].index
  df.drop(row1, axis=0, inplace=True)
  row2 = df[(df["frac_fluid_to_proppant_ratio"]==0)].index
  df.drop(row2, axis=0, inplace=True)
  row3 = df[(df["proppant_to_frac_fluid_ratio"]==0)].index
  df.drop(row3, axis=0, inplace=True)
  row4 = df[(df["proppant_to_frac_fluid_ratio"]>100)].index
  df.drop(row4, axis=0, inplace=True)
  return df

df3 = clean_data(df2)

<ipython-input-98-ffc17ec79752>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(row1, axis=0, inplace=True)
<ipython-input-98-ffc17ec79752>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(row2, axis=0, inplace=True)
<ipython-input-98-ffc17ec79752>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(row3, axis=0, inplace=True)
<ipython-input-98-ffc17ec79752>:10: SettingWithCopyWarning: 
A value is trying to be set

In [99]:
#calculating remaining null value
df3.isna().sum()/len(df3[['OilPeakRate']])


surface_x                       0.000000
surface_y                       0.000000
gross_perforated_length         0.008185
total_fluid                     0.074600
true_vertical_depth             0.005440
ffs_frac_type                   0.258768
proppant_intensity              0.074289
frac_fluid_intensity            0.076931
proppant_to_frac_fluid_ratio    0.093509
frac_fluid_to_proppant_ratio    0.093509
bin_lateral_length              0.008185
relative_well_position          0.000000
batch_frac_classification       0.000000
well_family_relationship        0.000000
frac_seasoning                  0.250065
OilPeakRate                     0.000000
dtype: float64

In [100]:
#Removing null value for numeric values
def fill_nan_1(df3):
  #Fill in NaN Values with mean after checking for outliers and spread of data - bin_lateral_length, gross_perforated_length
  df3['bin_lateral_length'].fillna(value=df3.bin_lateral_length.mean(), inplace=True)
  df3['gross_perforated_length'].fillna(value=df3.gross_perforated_length.mean(), inplace=True)
  df3['proppant_intensity'].fillna(value=df3.proppant_intensity.mean(), inplace=True)
  df3['total_fluid'].fillna(value=df3.total_fluid.mean(), inplace=True)
  df3['frac_fluid_to_proppant_ratio'].fillna(value=df3.frac_fluid_to_proppant_ratio.mean(), inplace=True)
  df3['proppant_to_frac_fluid_ratio'].fillna(value=df3.proppant_to_frac_fluid_ratio.mean(), inplace=True)
  df3['true_vertical_depth'].fillna(value=df3.true_vertical_depth.mean(), inplace=True)

  #Replace with mode as data RHS skewed
  df3['frac_fluid_intensity'].fillna(value=df3.frac_fluid_intensity.median(), inplace=True)
  return df3

df4 = fill_nan_1(df3)

<ipython-input-100-922fb719b7c1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bin_lateral_length'].fillna(value=df3.bin_lateral_length.mean(), inplace=True)
<ipython-input-100-922fb719b7c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['gross_perforated_length'].fillna(value=df3.gross_perforated_length.mean(), inplace=True)
<ipython-input-100-922fb719b7c1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['proppant_i

In [101]:
df4.isna().sum()/len(df4[['OilPeakRate']])

surface_x                       0.000000
surface_y                       0.000000
gross_perforated_length         0.000000
total_fluid                     0.000000
true_vertical_depth             0.000000
ffs_frac_type                   0.258768
proppant_intensity              0.000000
frac_fluid_intensity            0.000000
proppant_to_frac_fluid_ratio    0.000000
frac_fluid_to_proppant_ratio    0.000000
bin_lateral_length              0.000000
relative_well_position          0.000000
batch_frac_classification       0.000000
well_family_relationship        0.000000
frac_seasoning                  0.250065
OilPeakRate                     0.000000
dtype: float64

### Using clustering to fillna for the 25% variables

In [102]:
df_cluster = df4.drop(["ffs_frac_type", "frac_seasoning"], axis=1)
df_cluster = df_cluster.select_dtypes(exclude='object')
#normalising the cluster df
scalar = MinMaxScaler()
df_cluster_scale = scalar.fit_transform(df_cluster)

In [103]:
#num_clusters = 6 as it is the number of objects
clust = KMeans(n_clusters=6)
clust.fit(df_cluster_scale)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=6)

In [104]:
clust.labels_

array([4, 4, 4, ..., 1, 1, 1], dtype=int32)

In [105]:
df_cluster["cluster"] = clust.labels_
df_cluster

,surface_x,surface_y,gross_perforated_length,total_fluid,true_vertical_depth,proppant_intensity,frac_fluid_intensity,proppant_to_frac_fluid_ratio,frac_fluid_to_proppant_ratio,bin_lateral_length,OilPeakRate,cluster
0,1.383494e+06,7.173292e+05,3963.582677,3.900855e+06,8712.598425,19.174572,570.930579,0.224015,5.586810,1.0,46.623023,4
1,1.380344e+06,7.119163e+05,3179.133858,5.610964e+05,8627.952756,35.544272,176.493480,0.201391,4.965455,1.0,59.750009,4
2,1.379916e+06,7.178450e+05,3810.039370,1.621424e+06,8801.181102,75.214821,425.566078,0.176741,5.658008,1.0,10.785716,4
3,1.383567e+06,7.066402e+05,3723.425197,4.377134e+05,8635.826772,28.704465,117.556659,0.244176,4.095414,1.0,123.797638,4
4,1.378963e+06,7.026240e+05,3811.023622,6.229802e+05,8664.370079,29.595648,163.467933,0.181049,5.523377,1.0,102.309540,4
...,...,...,...,...,...,...,...,...,...,...,...,...
29432,9.477118e+05,1.057502e+06,4101.377953,5.499711e+05,8877.952756,23.371936,134.094222,0.174295,5.737403,1.0,162.307168,1
29433,9.593475e+05,1.033564e+06,4512.795276,3.900855e+06,8479.330709,108.575936,570.930579,0.224015,5.586810,1.0,171.071455,1
29434,9.669162e+05,1.057804e+06,3660.433071,6.214390e+05,8640.748031,24.001844,169.771992,0.141377,7.073290,1.0,7.171827,1
29435,9.462629e+05,1.034169e+06,5930.118110,1.956479e+06,8389.763780,64.730101,329.922425,0.196198,5.096893,1.0,336.044270,1


In [106]:
merged_df = df4.merge(df_cluster, how='outer')
df_cols = merged_df[["ffs_frac_type", "cluster"]]
result = df_cols.groupby("cluster")["ffs_frac_type"].apply(lambda x: x.mode().iloc[0]).reset_index()
result_dict = dict(zip(result["cluster"].to_numpy(), result['ffs_frac_type'].to_numpy()))
result_dict

{0: 'Slickwater Crosslink Hybrid',
 1: 'Slickwater Crosslink Hybrid',
 2: 'Slickwater Only',
 3: 'Slickwater Only',
 4: 'Slickwater Crosslink Hybrid',
 5: 'Slickwater Only'}

In [107]:
#merged_df["ffs_frac_type"].fillna(result)
merged_df.ffs_frac_type = merged_df.ffs_frac_type.fillna(merged_df.cluster.map(result_dict))

In [108]:
df_cols1 = merged_df[["frac_seasoning", "cluster"]]
result1 = df_cols1.groupby("cluster")["frac_seasoning"].mean()
result_dict1 = dict(result1)

In [109]:
merged_df.frac_seasoning = merged_df.frac_seasoning.fillna(merged_df.cluster.map(result_dict1))

In [110]:
df5 = merged_df.drop(['cluster'], axis=1)

In [111]:
df5.isna().sum()/len(df4[['OilPeakRate']])

surface_x                       0.0
surface_y                       0.0
gross_perforated_length         0.0
total_fluid                     0.0
true_vertical_depth             0.0
ffs_frac_type                   0.0
proppant_intensity              0.0
frac_fluid_intensity            0.0
proppant_to_frac_fluid_ratio    0.0
frac_fluid_to_proppant_ratio    0.0
bin_lateral_length              0.0
relative_well_position          0.0
batch_frac_classification       0.0
well_family_relationship        0.0
frac_seasoning                  0.0
OilPeakRate                     0.0
dtype: float64

In [112]:
#label encoding
def encode_cat_cols(df_val):
  df = df_val.copy()
  label_encoder = LabelEncoder()
  df['relative_well_position']= label_encoder.fit_transform(df['relative_well_position'])
  df['well_family_relationship']= label_encoder.fit_transform(df['well_family_relationship'])
  df['batch_frac_classification']= label_encoder.fit_transform(df['batch_frac_classification'])
  frac_type_mapping = {
      'Slickwater Only': 0,
      'Slickwater Linear Hybrid': 1,
      'Linear Only': 2,
      'Slickwater Crosslink Hybrid': 3,
      'Crosslink Only': 4,
      'Undefined': 5
  }

  df['ffs_frac_type_enc'] = df['ffs_frac_type'].map(frac_type_mapping)
  df = df.drop('ffs_frac_type', axis = 1)
  return df


df6 = encode_cat_cols(df5)

In [113]:
df6.isna().sum()
#df5['ffs_frac_type'].unique()

surface_x                       0
surface_y                       0
gross_perforated_length         0
total_fluid                     0
true_vertical_depth             0
proppant_intensity              0
frac_fluid_intensity            0
proppant_to_frac_fluid_ratio    0
frac_fluid_to_proppant_ratio    0
bin_lateral_length              0
relative_well_position          0
batch_frac_classification       0
well_family_relationship        0
frac_seasoning                  0
OilPeakRate                     0
ffs_frac_type_enc               0
dtype: int64

## Model Linear regression

In [114]:
X = df6.drop(['OilPeakRate'], axis=1)
y = df6['OilPeakRate']

In [115]:
#split train, test, val
Xtrain_val, Xtest, ytrain_val, ytest = train_test_split(X, y, test_size=0.1, random_state=42)
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.11, random_state=42)

In [116]:
#normalize the data
reg_scalar = StandardScaler()
reg_scalar.fit(Xtrain)
Xtrain = reg_scalar.transform(Xtrain)
Xval = reg_scalar.transform(Xval)
Xtest = reg_scalar.transform(Xtest)

In [117]:
Xtrain

array([[ 1.50364030e+00,  1.07814276e+00,  6.19452531e-02, ...,
        -3.48887553e-01,  2.10649441e-01, -1.13389553e+00],
       [ 3.16743249e-01,  4.94961168e-01, -6.89903610e-01, ...,
        -3.48887553e-01, -6.41380783e-01, -1.13389553e+00],
       [ 2.42476250e+00,  1.67271322e+00, -8.23147936e-01, ...,
         2.17099442e+00,  4.57961786e-04,  7.49364992e-01],
       ...,
       [ 3.48116307e+00, -2.30138770e-01, -6.04276378e-01, ...,
        -1.60882854e+00, -4.51821434e-02,  7.49364992e-01],
       [ 1.38771240e+00, -1.22237718e+00, -8.14376366e-01, ...,
        -1.60882854e+00, -2.42558125e-01,  1.37711850e+00],
       [-1.33009745e+00, -5.50928062e-01, -9.34672184e-01, ...,
        -1.60882854e+00, -4.60097757e-01, -5.06142025e-01]])

In [118]:
model = Linr()
model.fit(Xtrain, ytrain)

LinearRegression()

In [119]:
ytest_hat = model.predict(Xtest)

In [120]:
mse(ytest, ytest_hat, squared=False)

110.34272150002671

## CATBoost

In [121]:
Xc = df5.drop(['OilPeakRate'], axis=1)
yc = df5['OilPeakRate']

In [122]:
cat_df = df5.select_dtypes(include='object')
cat_features = list(cat_df.columns)
cat_features

['ffs_frac_type',
 'relative_well_position',
 'batch_frac_classification',
 'well_family_relationship']

In [123]:
#split train, test, val
Xctrain_val, Xctest, yctrain_val, yctest = train_test_split(Xc, yc, test_size=0.1, random_state=42)
Xctrain, Xcval, yctrain, ycval = train_test_split(Xctrain_val, yctrain_val, test_size=0.11, random_state=42)

In [124]:
eval_set = Xcval.insert(len(Xcval.columns), ycval.name, ycval.values)

In [125]:
model = cbr(cat_features=cat_features, random_seed=42)
model.fit(Xctrain, yctrain, eval_set=eval_set )

Learning rate set to 0.063107
0:	learn: 152.7265442	total: 41.8ms	remaining: 41.8s
1:	learn: 149.4592415	total: 88.8ms	remaining: 44.3s
2:	learn: 146.5788744	total: 140ms	remaining: 46.4s
3:	learn: 143.9033903	total: 173ms	remaining: 43.1s
4:	learn: 141.5456611	total: 213ms	remaining: 42.4s
5:	learn: 139.4718265	total: 253ms	remaining: 41.9s
6:	learn: 137.3454930	total: 292ms	remaining: 41.4s
7:	learn: 135.4791965	total: 333ms	remaining: 41.3s
8:	learn: 133.6696329	total: 372ms	remaining: 41s
9:	learn: 131.9228394	total: 438ms	remaining: 43.3s
10:	learn: 130.4462340	total: 494ms	remaining: 44.4s
11:	learn: 129.1058704	total: 568ms	remaining: 46.7s
12:	learn: 127.8366829	total: 640ms	remaining: 48.6s
13:	learn: 126.7148431	total: 735ms	remaining: 51.8s
14:	learn: 125.5343470	total: 818ms	remaining: 53.7s
15:	learn: 124.5698262	total: 892ms	remaining: 54.9s
16:	learn: 123.6132617	total: 956ms	remaining: 55.3s
17:	learn: 122.6340724	total: 1.01s	remaining: 55.3s
18:	learn: 121.7941020	tot

In [126]:
yctest_hat = model.predict(Xctest)

In [127]:
mse(yctest, yctest_hat, squared=False)

91.1475707560453

## Testing

In [133]:
test_df = pd.read_csv('/content/drive/MyDrive/Datathon 2024/Chevron/Test/scoring.csv')

In [134]:
#Data cleaning
test_df1 = remove_columns(test_df)
test_df2 = clean_data(test_df1)
test_df3 = fill_nan_1(test_df2)

In [135]:
test_df3.isnull().sum()

surface_x                       0
surface_y                       0
gross_perforated_length         0
total_fluid                     0
true_vertical_depth             0
ffs_frac_type                   0
proppant_intensity              0
frac_fluid_intensity            0
proppant_to_frac_fluid_ratio    0
frac_fluid_to_proppant_ratio    0
bin_lateral_length              0
relative_well_position          0
batch_frac_classification       0
well_family_relationship        0
frac_seasoning                  0
dtype: int64

In [136]:
submission = model.predict(test_df3)

In [143]:
submission = pd.DataFrame(submission, columns=['OilPeaks'])
submission.to_csv('/content/drive/MyDrive/Datathon 2024/Chevron/Test/submission_catboost.csv')

## Hyperparameter tuning

In [137]:
cat_train_pool = Pool(Xctrain, yctrain, cat_features=cat_features)
cat_val_pool = Pool(Xcval, ycval, cat_features=cat_features)
cat_test_pool = Pool(Xctest, yctest, cat_features=cat_features)

In [139]:
grid = {
    'learning_rate': [0.03, 1],
    'depth':[3, 6, 9, 12],
    'l2_leaf_reg': [2, 3, 4],
    'boosting_type': ['Ordered', 'Plain']
}

grid_search_model = cbr(cat_features=cat_features ,iterations=1000, random_seed=42)
grid_search_result = grid_search_model.grid_search(grid, cat_train_pool, verbose=20)

Streaming output truncated to the last 5000 lines.
986:	learn: 86.9287506	test: 108.2522381	best: 108.2522381 (986)	total: 2m 46s	remaining: 2.19s
987:	learn: 86.9251582	test: 108.2508539	best: 108.2508539 (987)	total: 2m 46s	remaining: 2.02s
988:	learn: 86.8977880	test: 108.2387053	best: 108.2387053 (988)	total: 2m 46s	remaining: 1.85s
989:	learn: 86.8947723	test: 108.2383717	best: 108.2383717 (989)	total: 2m 47s	remaining: 1.69s
990:	learn: 86.8794990	test: 108.2353403	best: 108.2353403 (990)	total: 2m 47s	remaining: 1.52s
991:	learn: 86.8509136	test: 108.2195326	best: 108.2195326 (991)	total: 2m 47s	remaining: 1.35s
992:	learn: 86.8504486	test: 108.2205453	best: 108.2195326 (991)	total: 2m 47s	remaining: 1.18s
993:	learn: 86.8492046	test: 108.2218759	best: 108.2195326 (991)	total: 2m 47s	remaining: 1.01s
994:	learn: 86.8135235	test: 108.2060873	best: 108.2060873 (994)	total: 2m 47s	remaining: 843ms
995:	learn: 86.8127869	test: 108.2071185	best: 108.2060873 (994)	total: 2m 47s	remain

KeyboardInterrupt: 